<a href="https://colab.research.google.com/github/bay195/Waste-Classification/blob/main/WasteClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#UAS Machine Learning
# **Kelompok 6**
1. Bayu Unggul Sejati
2. Hengky Triyo
3. Salwa Salsabila
4. Rafi Albar Kurniawan

In [ ]:
pip install flask

In [ ]:
pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.applications.efficientnet import EfficientNetB1, preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import zipfile
import os
import flask

In [ ]:
data_url = 'https://github.com/bay195/Waste-Classification/raw/main/Anorganik.zip'
urllib.request.urlretrieve(data_url, 'Anorganik.zip')
local_file = 'Anorganik.zip'
zip_ref = zipfile.ZipFile(local_file, 'r')
zip_ref.extractall('data/')
zip_ref.close()

In [ ]:
BASE_DIR = 'data/Anorganik'
train_dir = os.path.join(BASE_DIR, 'Train')
validation_dir = os.path.join(BASE_DIR, 'Validation')

In [ ]:
# Define image generators
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# Use the correct target size for EfficientNet (e.g., 224x224 for EfficientNetB0)
target_size = (224, 224)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size=32,
    class_mode='categorical',
    target_size=target_size
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    batch_size=32,
    class_mode='categorical',
    target_size=target_size
)

Found 2226 images belonging to 6 classes.
Found 955 images belonging to 6 classes.


In [ ]:
# Load the EfficientNetB0 model without the top classification layer
base_model = EfficientNetB1(include_top=False, input_shape=(224, 224, 3))

27018416/27018416 [==============================] - 0s 0us/step


In [ ]:
# Freeze the base model
base_model.trainable = False

In [ ]:
# Create a new model on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(6, activation='softmax')  # For multi-class classification with 6 categories
])

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',  # Use categorical crossentropy for multi-class classification
              metrics=['accuracy'])

In [ ]:
# Define the callbacks
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True, monitor='val_loss', mode='min'),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10,
    callbacks=callbacks
)

Epoch 1/10
69/69 [==============================] - ETA: 0s - loss: 0.7579 - accuracy: 0.7830

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


69/69 [==============================] - 387s 5s/step - loss: 0.7579 - accuracy: 0.7830 - val_loss: 0.3073 - val_accuracy: 0.9364
Epoch 2/10
69/69 [==============================] - 411s 6s/step - loss: 0.2992 - accuracy: 0.9280 - val_loss: 0.2024 - val_accuracy: 0.9494
Epoch 3/10
69/69 [==============================] - 357s 5s/step - loss: 0.2509 - accuracy: 0.9262 - val_loss: 0.1730 - val_accuracy: 0.9494
Epoch 4/10
69/69 [==============================] - 406s 6s/step - loss: 0.2023 - accuracy: 0.9412 - val_loss: 0.1532 - val_accuracy: 0.9591
Epoch 5/10
69/69 [==============================] - 400s 6s/step - loss: 0.1884 - accuracy: 0.9467 - val_loss: 0.1448 - val_accuracy: 0.9634
Epoch 6/10
69/69 [==============================] - 354s 5s/step - loss: 0.1563 - accuracy: 0.9581 - val_loss: 0.1386 - val_accuracy: 0.9569
Epoch 7/10
69/69 [==============================] - 360s 5s/step - loss: 0.1385 - accuracy: 0.9640 - val_loss: 0.1345 - val_accuracy: 0.9612
Epoch 8/10
69/69 [======

In [ ]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

30/30 [==============================] - 104s 3s/step - loss: 0.1245 - accuracy: 0.9623
Validation Loss: 0.12454579025506973
Validation Accuracy: 0.962303638458252


In [ ]:
model.save("efficiennetB1.h5")

In [ ]:
model = load_model("efficiennetB1.h5")

In [ ]:
# Load and preprocess uploaded image
img_path = 'test5.jpg'  # Ganti dengan path gambar yang diunggah
img = image.load_img(img_path, target_size=(224, 224))  # Sesuaikan dengan ukuran input model Anda
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

# Use preprocess_input function for EfficientNet
img_array = preprocess_input(img_array)

# Predict class
prediction = model.predict(img_array)

# Display result
print(prediction)

1/1 [==============================] - 0s 125ms/step
[[0.00875414 0.16312107 0.00166622 0.02295705 0.00333665 0.8001648 ]]


In [ ]:
# Memperoleh nama folder (kelas) dari generator pelatihan
class_names_train = train_generator.class_indices
print("Nama folder (kelas) dari generator pelatihan:", class_names_train)

# Memperoleh nama folder (kelas) dari generator validasi
class_names_validation = validation_generator.class_indices
print("Nama folder (kelas) dari generator validasi:", class_names_validation)

In [ ]:
# Kamus yang memetakan indeks ke nama folder kategori
class_names = {'BotolKaca': 0, 'BotolPlastik': 1, 'GelasDisposable': 2, 'Kaleng': 3, 'Kardus': 4, 'WadahKaca': 5}

# Output probabilitas dari model
output_probabilities = prediction

# Konversi output probabilitas ke dalam array NumPy
output_probabilities_array = np.array(output_probabilities)

# Mendapatkan indeks dari kelas dengan probabilitas tertinggi
predicted_class_index = np.argmax(output_probabilities_array)

# Mendapatkan nama folder kategori berdasarkan indeks
predicted_class_name = [name for name, index in class_names.items() if index == predicted_class_index][0]

# Tampilkan nama folder kategori yang diprediksi
print("Folder kategori yang diprediksi:", predicted_class_name)


Folder kategori yang diprediksi: WadahKaca


In [ ]:
# Unfreeze the base model for fine-tuning
base_model.trainable = True

In [ ]:
# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Continue training the model (fine-tuning)
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10,  # Additional epochs for fine-tuning
    callbacks=callbacks
)

In [ ]:
model.save("finedefficientnetB0.h5")

In [ ]:
# Load and preprocess uploaded image
img_path = 'test1.jpg'  # Ganti dengan path gambar yang diunggah
img = image.load_img(img_path, target_size=(224, 224))  # Sesuaikan dengan ukuran input model Anda
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

# Use preprocess_input function for EfficientNet
img_array = preprocess_input(img_array)

# Predict class
prediction = model.predict(img_array)

# Display result
print(prediction)

In [ ]:
# Kamus yang memetakan indeks ke nama folder kategori
class_names = {'BotolKaca': 0, 'BotolPlastik': 1, 'GelasDisposable': 2, 'Kaleng': 3, 'Kardus': 4, 'WadahKaca': 5}

# Output probabilitas dari model
output_probabilities = prediction

# Konversi output probabilitas ke dalam array NumPy
output_probabilities_array = np.array(output_probabilities)

# Mendapatkan indeks dari kelas dengan probabilitas tertinggi
predicted_class_index = np.argmax(output_probabilities_array)

# Mendapatkan nama folder kategori berdasarkan indeks
predicted_class_name = [name for name, index in class_names.items() if index == predicted_class_index][0]

# Tampilkan nama folder kategori yang diprediksi
print("Folder kategori yang diprediksi:", predicted_class_name)

In [ ]:
# Load the best model
model.load_weights("best_model.h5")